In [30]:
import pandas as pd
import numpy as np
import warnings
import jieba
import jieba.posseg as pseg
jieba.load_userdict('../source/words/problem.txt')
import re
from gensim.models import Word2Vec
from bayes_opt import BayesianOptimization
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sqlalchemy import create_engine
import sys
sys.path.extend(['../src/'])
sys.path.extend(['../bert-utils/'])
from config import *
config = Config()

In [31]:
def data_sql_load():
    engine = create_engine('mysql+pymysql://%s:%s@%s:3306/%s?charset=utf8' %(config.sql_user, config.sql_password, config.sql_ip, config.sql_database),echo = False)
    sql = "select * from fe_data2"
    data = pd.read_sql_query(sql, engine)
    return data

data = data_sql_load()

In [41]:
from bert_serving.client import BertClient

bc = BertClient()
output = bc.encode(list(data['jieba']), show_tokens=True)

C:\Users\kenshinpg\Anaconda3\lib\site-packages\bert_serving\client\__init__.py:286: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)
C:\Users\kenshinpg\Anaconda3\lib\site-packages\bert_serving\client\__init__.py:295: UserWarning: "show_tokens=True", but the server does not support showing tokenization info to clients.
here is what you can do:
- start a new server with "bert-serving-start -show_tokens_to_client ..."
- or, use "encode(show_tokens=False)"
  warnings.warn('"show_tokens=True", but the server does not support showing tokenization info to clients.\n'


In [40]:
help(bc.encode)

Help on method encode in module bert_serving.client:

encode(texts, blocking=True, is_tokenized=False, show_tokens=False) method of bert_serving.client.BertClient instance
    Encode a list of strings to a list of vectors
    
    `texts` should be a list of strings, each of which represents a sentence.
    If `is_tokenized` is set to True, then `texts` should be list[list[str]],
    outer list represents sentence and inner list represent tokens in the sentence.
    Note that if `blocking` is set to False, then you need to fetch the result manually afterwards.
    
    .. highlight:: python
    .. code-block:: python
    
        with BertClient() as bc:
            # encode untokenized sentences
            bc.encode(['First do it',
                      'then do it right',
                      'then do it better'])
    
            # encode tokenized sentences
            bc.encode([['First', 'do', 'it'],
                       ['then', 'do', 'it', 'right'],
                       [

In [38]:
output = pd.DataFrame(output)

output.columns = [f'bert_{i}' for i in range(output.shape[1])]

In [39]:
output

,bert_0,bert_1,bert_2,bert_3,bert_4,bert_5,bert_6,bert_7,bert_8,bert_9,...,bert_758,bert_759,bert_760,bert_761,bert_762,bert_763,bert_764,bert_765,bert_766,bert_767
0,0.425209,0.272579,0.128432,-0.372426,0.514865,-0.831870,0.246775,-0.898366,-0.007850,0.225695,...,-0.000721,0.111907,0.099297,0.251489,-0.484872,0.273891,-0.527948,0.255248,-0.466930,-0.170265
1,0.425209,0.272579,0.128432,-0.372426,0.514865,-0.831870,0.246775,-0.898366,-0.007850,0.225695,...,-0.000721,0.111907,0.099297,0.251489,-0.484872,0.273891,-0.527948,0.255248,-0.466930,-0.170265
2,0.205017,0.176673,0.092066,-0.567462,0.107969,-0.759664,0.272738,-0.260807,0.071863,0.449763,...,-0.224904,-0.060605,0.498670,0.232057,-0.203619,-0.015776,-0.318980,-0.247616,-0.778913,0.196067
3,0.210913,0.206218,-0.644260,-0.429313,0.344572,-1.722605,0.319022,0.201494,-0.027946,-0.069849,...,0.006096,-0.101833,0.035319,-0.613570,0.059524,0.018598,-0.141281,0.008946,-0.525078,-0.297281
4,0.392538,0.236353,-0.010159,-0.445804,-0.011537,-0.447266,0.174687,-0.405778,0.256162,0.418907,...,-0.006404,0.373825,0.391858,0.372949,-0.285549,-0.149692,-0.407858,0.275736,-0.340822,-0.402884
5,-0.036571,0.258302,-0.433142,-0.184532,0.105609,-1.072161,0.026170,-0.054712,-0.140323,0.140110,...,0.063524,-0.334734,0.474786,0.156826,-0.097121,0.481840,0.350756,0.009057,-0.549791,0.333796
6,0.191909,0.049274,-0.305502,-0.560318,0.502294,-0.646202,0.271351,-0.414150,-0.216184,0.211570,...,0.101525,-0.377428,0.406256,-0.030747,-0.113613,0.081503,0.016744,0.090690,-0.726486,-0.226437
7,0.560336,-0.306933,-0.198498,-0.462826,0.144725,-1.285107,0.207088,-0.154079,-0.048897,0.223550,...,-0.308721,-0.158078,-0.107999,0.011836,0.064854,0.115062,-1.022639,0.361352,-0.466934,0.218813
8,0.159307,-0.093388,-0.398291,-0.456278,0.246761,-0.987667,0.204126,-0.214935,0.029188,0.187744,...,-0.105741,-0.291409,-0.258674,-0.423103,0.053660,0.149732,-0.137925,0.056144,-0.671396,-0.250189
9,-0.175602,0.051621,-0.350513,-0.159706,0.169565,-0.909228,-0.116146,-0.278692,-0.204426,0.356223,...,0.000711,-0.401829,0.269507,0.316610,0.049968,0.177611,-0.246918,-0.205656,-1.095653,-0.132399


In [25]:
from sklearn.model_selection import train_test_split

def jieba_train_test_split(data_):
    data = data_[['jieba', '紧急程度']].copy()
    X_test = data.loc[data['紧急程度'] == '未知','jieba'].apply(lambda x: x.split())
    data_train = data.loc[data['紧急程度'] != '未知',:]
    y = data_train['紧急程度'].map({'紧急': '紧急或重要', '重要': '紧急或重要', '无要求': '不紧急', '正常': '不紧急'}).reset_index(drop = True)
    X = data_train['jieba'].apply(lambda x: x.split())
    X_train, X_valid, Y_train, Y_valid = train_test_split(X, y, test_size = 0.3, random_state= 2019, stratify = y)
    return list(X_train), list(X_valid), list(X_test), list(Y_train), list(Y_valid)

X_train, X_valid, X_test, Y_train, Y_valid = jieba_train_test_split(data)

In [18]:
from kashgari.tasks.classification import BLSTMModel
from kashgari.embeddings import BERTEmbedding
embedding = BERTEmbedding('bert-base-chinese', sequence_length=30)

embedded_vector = embedding.embed(X_train[0])

[array([[ 100,  100,  100,  100,  100,  100,  100, 9046, 8230,  100,  100,
         100,  100,  100,  100,  100,  100,  100,  100,  100,  100,  100,
         100,  100, 3890,  100,  100, 1779,  100,  102]]), array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])]
